In [1]:
import pandas as pd
import numpy as np
import os.path
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression, LassoCV, RidgeCV, ElasticNetCV

from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import (cross_val_score, train_test_split, 
                                     KFold, GridSearchCV)
import scipy.stats as stats


from sklearn.metrics import r2_score, mean_squared_error

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [15]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import lars_path
from sklearn.linear_model import ElasticNet



In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
nba = pd.read_csv('nba_v3.csv')

In [4]:
nba.columns

Index(['Unnamed: 0', 'Name', 'Team', 'Age', 'Games', 'Games_Start', 'Min_PG',
       'Field_Goals', 'Field_Goals_Attempts', '3Point_Goals',
       '3Point_Goals_Attempts', '2Point_Goals', '2Point_Goals_Attempts',
       'Effective_Field_Goal_Per', 'Free_Throw', 'Free_Throw_Attempts',
       'Offensive_Rebounds', 'Defensive_Rebounds', 'Assists', 'Steals',
       'Blocks', 'Turnovers', 'Points', 'Position', 'Year', 'Salary'],
      dtype='object')

In [5]:
nba = nba.drop('Unnamed: 0', axis = 1)

In [8]:
nba['Salary'] = nba['Salary'] / (1000000) # Salary in Million

In [9]:
nba['Salary'].describe()

count    1106.000000
mean        7.576742
std         7.657705
min         0.004737
25%         1.967220
50%         4.148520
75%        10.830309
max        31.626953
Name: Salary, dtype: float64

In [10]:
#nba_v1= nba._get_numeric_data()

X, y = nba_v1.drop(columns = ['Salary']), nba_v1['Salary']

In [11]:
#Split the data into test/train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train

,Age,Games,Games_Start,Min_PG,Field_Goals,Field_Goals_Attempts,3Point_Goals,3Point_Goals_Attempts,2Point_Goals,2Point_Goals_Attempts,...,Free_Throw,Free_Throw_Attempts,Offensive_Rebounds,Defensive_Rebounds,Assists,Steals,Blocks,Turnovers,Points,Year
773,20,74,74,26.1,2.5,5.8,1.4,3.9,1.1,1.9,...,0.5,0.5,0.4,1.5,1.0,0.5,0.2,0.6,6.9,2019
728,33,67,61,30.2,4.8,10.6,1.5,4.2,3.3,6.4,...,0.9,0.9,1.5,5.3,4.0,0.8,0.9,1.2,11.9,2020
867,22,63,10,22.8,3.6,8.3,1.7,4.3,1.9,4.0,...,0.8,0.8,0.2,2.7,1.8,0.4,0.2,0.9,9.7,2019
507,34,53,53,33.1,7.4,15.0,1.2,3.0,6.2,12.0,...,3.0,3.0,1.9,5.5,2.4,0.7,1.6,1.4,18.9,2020
319,24,60,11,24.0,3.4,8.1,2.0,5.6,1.4,2.5,...,0.8,0.8,0.6,2.8,1.4,0.6,0.3,0.7,9.6,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,26,63,2,27.8,7.5,12.9,0.0,0.3,7.5,12.6,...,3.7,3.7,2.6,4.5,1.7,0.6,1.1,1.7,18.6,2020
121,22,42,0,20.9,4.1,9.5,2.0,5.0,2.1,4.5,...,1.4,1.4,0.3,2.1,2.1,0.5,0.1,1.3,11.7,2021
1044,28,80,80,35.5,8.5,19.2,3.0,8.0,5.6,11.1,...,5.9,5.9,0.9,3.8,6.9,1.1,0.4,2.7,25.8,2019
1095,31,51,6,10.4,1.5,3.1,0.2,0.8,1.3,2.3,...,0.6,0.6,0.9,2.0,1.2,0.3,0.3,0.9,3.9,2019


In [13]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

In [16]:
#Feature scaling for train, and test so that we can run our ridge model on each
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

In [20]:
#Feature transforms for train and test so that we can run our poly model on each
scaler = StandardScaler()
poly = PolynomialFeatures(degree=2) 

#X_train_sca_poly = poly.fit_transform(X_train_scaled)
#X_test_sca_poly = poly.transform(X_test_scaled)

param_poly =  {'lasso__alpha':np.logspace(-4, 2, 100)}

#Setting up a pipeline
pipe = Pipeline([('scaler', StandardScaler()),('poly', PolynomialFeatures(degree=2)), ('lasso', Lasso())])

#pipe.get_params().keys()

#setting up the grid search
grid_poly = GridSearchCV(pipe,param_poly,scoring='r2',cv=kfold)


#fitting grid search to training data
grid_result_poly = grid_poly.fit(X_train, y_train)

# View best parameter
print('Best Score: ', grid_result_poly.best_score_)
print('Best Param: ', grid_result_poly.best_params_)


Best Score:  0.5672438074435703
Best Param:  {'lasso__alpha': 0.14174741629268048}


In [22]:
grid_poly.best_estimator_.steps[2][1]

Lasso(alpha=0.14174741629268048)

In [26]:
print('Feature coefficient results for the Poly(degree= 2), Lasso with All Numerical Features: \n')
for feature, coef in zip(X_train.columns, grid_poly.best_estimator_.steps[2][1].coef_):
    print(feature, ':', f'{coef:.2f}') 



# score fit model on test data
print('R Square for the train data :', grid_result_poly.score(X_train, y_train))
print('R Square for the test data:', grid_result_poly.score(X_test, y_test))



Feature coefficient results for the Poly(degree= 2), Lasso with All Numerical Features: 

Age : 0.00
Games : 2.84
Games_Start : -0.35
Min_PG : 0.02
Field_Goals : 2.02
Field_Goals_Attempts : 0.00
3Point_Goals : 0.64
3Point_Goals_Attempts : 0.00
2Point_Goals : 0.00
2Point_Goals_Attempts : 0.00
Effective_Field_Goal_Per : 0.59
Free_Throw : -0.00
Free_Throw_Attempts : 0.00
Offensive_Rebounds : 0.00
Defensive_Rebounds : 0.05
Assists : 0.60
Steals : 0.00
Blocks : 0.00
Turnovers : 0.00
Points : 0.29
Year : 0.00
R Square for the train data : 0.6228689629137723
R Square for the test data: 0.5987907856832946


In [25]:
print('Feature coefficient results for the Poly(degree= 2), Lasso with All Numerical Features: \n')
for feature, coef in zip(grid_poly.best_estimator_.steps[1][1].get_feature_names(), grid_poly.best_estimator_.steps[2][1].coef_):
    print(feature, ':', f'{coef:.2f}') 



Feature coefficient results for the Poly(degree= 2), Lasso with All Numerical Features: 

1 : 0.00
x0 : 2.84
x1 : -0.35
x2 : 0.02
x3 : 2.02
x4 : 0.00
x5 : 0.64
x6 : 0.00
x7 : 0.00
x8 : 0.00
x9 : 0.59
x10 : -0.00
x11 : 0.00
x12 : 0.00
x13 : 0.05
x14 : 0.60
x15 : 0.00
x16 : 0.00
x17 : 0.00
x18 : 0.29
x19 : 0.00
x20 : 0.00
x0^2 : -0.15
x0 x1 : -0.28
x0 x2 : 0.44
x0 x3 : 0.00
x0 x4 : 0.00
x0 x5 : 0.00
x0 x6 : 0.14
x0 x7 : 0.00
x0 x8 : 0.00
x0 x9 : 0.00
x0 x10 : 0.00
x0 x11 : 0.50
x0 x12 : 0.00
x0 x13 : 0.00
x0 x14 : 0.92
x0 x15 : 0.53
x0 x16 : 0.12
x0 x17 : -0.00
x0 x18 : 0.00
x0 x19 : 0.04
x0 x20 : -0.14
x1^2 : 0.00
x1 x2 : -0.00
x1 x3 : 0.00
x1 x4 : 0.00
x1 x5 : 0.00
x1 x6 : -0.36
x1 x7 : -0.00
x1 x8 : 0.00
x1 x9 : 0.00
x1 x10 : -0.00
x1 x11 : 0.00
x1 x12 : 0.00
x1 x13 : 0.00
x1 x14 : -0.00
x1 x15 : 0.00
x1 x16 : 0.03
x1 x17 : 0.03
x1 x18 : -0.00
x1 x19 : 0.00
x1 x20 : -0.07
x2^2 : 0.00
x2 x3 : 0.00
x2 x4 : 0.00
x2 x5 : 0.00
x2 x6 : -0.00
x2 x7 : -0.00
x2 x8 : 0.63
x2 x9 : 0.00
x2 x10 : 

In [41]:
 y_test - grid_result_poly.predict(X_test)

323    -0.365977
732     9.467995
56      4.419797
693    -2.396795
485    -0.242041
          ...   
1070   -3.434379
581    -7.764823
1005    3.279658
1004    2.474844
528     7.160135
Name: Salary, Length: 222, dtype: float64

In [42]:
y_te = y_test.to_frame()
y_te['Predicted'] =  grid_result_poly.predict(X_test)
y_te['Error'] = y_te['Salary'] - y_te['Predicted']
y_te

,Salary,Predicted,Error
323,0.898310,1.264287,-0.365977
732,19.863636,10.395641,9.467995
56,14.500000,10.080203,4.419797
693,2.875000,5.271795,-2.396795
485,2.625000,2.867041,-0.242041
...,...,...,...
1070,0.705361,4.139740,-3.434379
581,2.320044,10.084867,-7.764823
1005,4.661280,1.381622,3.279658
1004,17.469565,14.994721,2.474844


In [44]:
test = pd.concat([y_te, X_test], axis=1)
test

,Salary,Predicted,Error,Age,Games,Games_Start,Min_PG,Field_Goals,Field_Goals_Attempts,3Point_Goals,...,Free_Throw,Free_Throw_Attempts,Offensive_Rebounds,Defensive_Rebounds,Assists,Steals,Blocks,Turnovers,Points,Year
323,0.898310,1.264287,-0.365977,20,30,0,6.4,0.8,2.2,0.2,...,0.4,0.4,0.4,0.7,0.3,0.2,0.1,0.2,2.3,2021
732,19.863636,10.395641,9.467995,24,62,62,32.5,5.4,12.4,1.2,...,2.4,2.4,1.7,5.9,3.7,0.8,0.6,1.6,14.4,2020
56,14.500000,10.080203,4.419797,34,63,1,21.6,4.2,10.0,1.7,...,1.3,1.3,0.7,4.1,1.4,0.7,0.6,1.0,11.4,2021
693,2.875000,5.271795,-2.396795,26,63,62,25.1,3.8,9.2,2.3,...,1.2,1.2,0.2,1.8,1.7,0.5,0.0,0.9,11.2,2020
485,2.625000,2.867041,-0.242041,24,44,0,11.0,1.4,3.6,0.6,...,0.5,0.5,0.3,1.3,1.0,0.3,0.2,0.8,3.9,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0.705361,4.139740,-3.434379,23,34,19,23.5,2.8,6.6,1.4,...,1.2,1.2,1.2,3.4,1.5,0.4,1.0,1.1,8.3,2019
581,2.320044,10.084867,-7.764823,30,40,37,23.1,4.4,10.7,2.0,...,1.6,1.6,0.3,1.4,3.7,0.3,0.2,1.9,12.2,2020
1005,4.661280,1.381622,3.279658,21,46,27,18.0,1.9,4.3,0.5,...,0.7,0.7,0.7,3.2,1.2,0.4,0.5,0.8,5.0,2019
1004,17.469565,14.994721,2.474844,28,66,66,31.3,2.8,6.4,0.7,...,1.0,1.0,0.9,6.4,6.9,1.4,1.1,2.6,7.4,2019


In [46]:
test_2021 = test[test['Year'] ==2021]
test_2021

,Salary,Predicted,Error,Age,Games,Games_Start,Min_PG,Field_Goals,Field_Goals_Attempts,3Point_Goals,...,Free_Throw,Free_Throw_Attempts,Offensive_Rebounds,Defensive_Rebounds,Assists,Steals,Blocks,Turnovers,Points,Year
323,0.898310,1.264287,-0.365977,20,30,0,6.4,0.8,2.2,0.2,...,0.4,0.4,0.4,0.7,0.3,0.2,0.1,0.2,2.3,2021
56,14.500000,10.080203,4.419797,34,63,1,21.6,4.2,10.0,1.7,...,1.3,1.3,0.7,4.1,1.4,0.7,0.6,1.0,11.4,2021
86,4.862040,2.372529,2.489511,22,62,0,11.0,1.7,3.4,0.4,...,0.3,0.3,0.4,1.8,0.5,0.3,0.2,0.4,4.1,2021
218,2.320044,6.677853,-4.357809,31,36,4,15.2,3.5,6.2,0.0,...,1.1,1.1,1.7,4.3,0.6,0.3,1.3,1.1,8.1,2021
88,10.865952,13.470483,-2.604531,27,42,31,30.4,6.7,13.5,2.8,...,3.4,3.4,0.5,2.5,1.8,1.1,0.2,1.8,19.6,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,13.013700,8.139035,4.874665,36,31,0,18.6,2.8,6.9,1.6,...,1.5,1.5,0.1,1.6,1.3,0.3,0.1,0.8,8.7,2021
12,8.817143,2.636045,6.181098,24,70,9,17.5,2.5,5.9,0.6,...,0.6,0.6,0.3,1.7,3.7,0.9,0.1,0.7,6.3,2021
355,2.345640,3.385650,-1.040010,24,66,8,17.8,2.6,6.6,1.0,...,1.0,1.0,0.2,1.1,1.9,0.7,0.2,1.0,7.2,2021
184,1.517981,2.935673,-1.417692,21,42,0,18.1,3.4,8.0,1.2,...,1.1,1.1,0.5,1.8,1.5,0.5,0.3,0.7,9.0,2021


In [48]:
df = test_2021.sort_values(by = ['Error'], ascending=True).head(15)
df

,Salary,Predicted,Error,Age,Games,Games_Start,Min_PG,Field_Goals,Field_Goals_Attempts,3Point_Goals,...,Free_Throw,Free_Throw_Attempts,Offensive_Rebounds,Defensive_Rebounds,Assists,Steals,Blocks,Turnovers,Points,Year
128,8.049360,17.073382,-9.024022,21,66,66,34.3,9.8,20.5,2.9,...,5.2,5.2,0.8,7.2,8.6,1.0,0.5,4.3,27.7,2021
23,6.571800,13.002879,-6.431079,22,63,63,33.7,7.7,17.7,2.2,...,7.7,7.7,0.6,3.3,9.4,0.8,0.2,4.1,25.3,2021
39,3.500000,9.235856,-5.735856,28,69,3,26.0,4.0,7.1,0.2,...,0.5,0.5,0.8,2.9,6.6,1.9,0.3,2.0,8.6,2021
109,2.283034,7.243482,-4.960448,29,35,0,18.4,3.4,7.6,1.9,...,0.9,0.9,0.5,3.2,0.8,0.2,0.3,0.6,9.7,2021
139,3.562178,8.389591,-4.827413,25,68,32,28.7,4.9,11.1,2.0,...,1.0,1.0,1.6,4.5,1.9,0.5,0.6,1.7,12.9,2021
231,4.767000,9.306727,-4.539727,34,56,25,27.3,2.8,6.8,1.2,...,0.7,0.7,0.5,2.3,2.2,0.8,0.5,1.0,7.6,2021
218,2.320044,6.677853,-4.357809,31,36,4,15.2,3.5,6.2,0.0,...,1.1,1.1,1.7,4.3,0.6,0.3,1.3,1.1,8.1,2021
83,7.969537,12.258868,-4.289331,35,32,32,30.0,1.5,4.1,0.8,...,0.6,0.6,1.0,3.6,1.4,0.9,0.6,1.0,4.4,2021
168,3.491159,7.713320,-4.222161,25,47,4,28.7,3.2,7.3,1.3,...,1.5,1.5,1.1,6.9,2.3,0.8,0.3,1.0,9.2,2021
289,3.717000,7.734149,-4.017149,27,36,23,25.9,3.1,7.6,1.5,...,1.1,1.1,0.5,3.2,1.9,0.6,0.4,1.0,8.8,2021


In [68]:
df.describe()

,Salary,Predicted,Error,Age,Games,Games_Start,Min_PG,Field_Goals,Field_Goals_Attempts,3Point_Goals,...,Free_Throw,Free_Throw_Attempts,Offensive_Rebounds,Defensive_Rebounds,Assists,Steals,Blocks,Turnovers,Points,Year
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,...,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.0
mean,3.863318,8.539762,-4.676444,27.400000,50.666667,24.333333,25.493333,3.940000,8.633333,1.406667,...,1.720000,1.720000,0.726667,3.580000,2.873333,0.793333,0.400000,1.426667,11.006667,2021.0
std,2.225817,3.377771,1.492697,4.067291,15.201817,23.457154,5.804489,2.141695,4.569881,0.810173,...,2.000786,2.000786,0.447958,1.593379,2.857938,0.409646,0.316228,1.192516,6.600051,0.0
min,0.250000,3.916702,-9.024022,21.000000,26.000000,0.000000,15.200000,1.500000,4.100000,0.000000,...,0.500000,0.500000,0.200000,1.800000,0.600000,0.200000,0.000000,0.500000,4.400000,2021.0
25%,2.301539,6.643250,-4.893930,25.000000,36.000000,4.000000,20.400000,2.950000,6.500000,0.900000,...,0.750000,0.750000,0.400000,2.650000,1.350000,0.550000,0.200000,0.750000,7.850000,2021.0
50%,3.562178,7.734149,-4.289331,28.000000,56.000000,22.000000,26.000000,3.400000,7.300000,1.400000,...,1.100000,1.100000,0.500000,3.200000,1.900000,0.800000,0.300000,1.000000,8.800000,2021.0
75%,4.483500,9.271292,-3.922487,29.000000,64.500000,32.000000,29.350000,3.900000,8.100000,1.950000,...,1.400000,1.400000,0.900000,3.950000,2.300000,0.950000,0.550000,1.400000,10.350000,2021.0
max,8.049360,17.073382,-2.985647,35.000000,69.000000,66.000000,34.300000,9.800000,20.500000,2.900000,...,7.700000,7.700000,1.700000,7.200000,9.400000,1.900000,1.300000,4.300000,27.700000,2021.0


In [75]:
df.reset_index(inplace=True)

In [81]:
 nba.reset_index(inplace=True)

In [82]:
underpaid = pd.merge(df, nba, how='left', on=['index'])
underpaid

,index,Salary_x,Predicted,Error,Age_x,Games_x,Games_Start_x,Min_PG_x,Field_Goals_x,Field_Goals_Attempts_x,...,Offensive_Rebounds_y,Defensive_Rebounds_y,Assists_y,Steals_y,Blocks_y,Turnovers_y,Points_y,Position,Year_y,Salary_y
0,128,8.049360,17.073382,-9.024022,21,66,66,34.3,9.8,20.5,...,0.8,7.2,8.6,1.0,0.5,4.3,27.7,Point Guard and Shooting Guard,2021,8.049360
1,23,6.571800,13.002879,-6.431079,22,63,63,33.7,7.7,17.7,...,0.6,3.3,9.4,0.8,0.2,4.1,25.3,Point Guard,2021,6.571800
2,39,3.500000,9.235856,-5.735856,28,69,3,26.0,4.0,7.1,...,0.8,2.9,6.6,1.9,0.3,2.0,8.6,Point Guard,2021,3.500000
3,109,2.283034,7.243482,-4.960448,29,35,0,18.4,3.4,7.6,...,0.5,3.2,0.8,0.2,0.3,0.6,9.7,Center and Power Forward,2021,2.283034
4,139,3.562178,8.389591,-4.827413,25,68,32,28.7,4.9,11.1,...,1.6,4.5,1.9,0.5,0.6,1.7,12.9,Power Forward and Small Forward,2021,3.562178
5,231,4.767000,9.306727,-4.539727,34,56,25,27.3,2.8,6.8,...,0.5,2.3,2.2,0.8,0.5,1.0,7.6,"Shooting Guard, Small Forward, and Point Guard",2021,4.767000
6,218,2.320044,6.677853,-4.357809,31,36,4,15.2,3.5,6.2,...,1.7,4.3,0.6,0.3,1.3,1.1,8.1,Center,2021,2.320044
7,83,7.969537,12.258868,-4.289331,35,32,32,30.0,1.5,4.1,...,1.0,3.6,1.4,0.9,0.6,1.0,4.4,"Small Forward, Power Forward, and Shooting Guard",2021,7.969537
8,168,3.491159,7.713320,-4.222161,25,47,4,28.7,3.2,7.3,...,1.1,6.9,2.3,0.8,0.3,1.0,9.2,Shooting Guard and Small Forward,2021,3.491159
9,289,3.717000,7.734149,-4.017149,27,36,23,25.9,3.1,7.6,...,0.5,3.2,1.9,0.6,0.4,1.0,8.8,Small Forward and Shooting Guard,2021,3.717000


In [83]:
underpaid.Name

0          Luka Dončić
1           Trae Young
2       T.J. McConnell
3         Mike Muscala
4           Kyle Kuzma
5       Garrett Temple
6     Hassan Whiteside
7          P.J. Tucker
8            Josh Hart
9         Danuel House
10      Reggie Bullock
11           Raul Neto
12       Frank Jackson
13          Damion Lee
14    Maurice Harkless
Name: Name, dtype: object